# The `QuantumState` class

Express an N-qubit quantum state as an array of N columns, where rows are binary bit values, complemented by a vector of coefficients. Note the similarity with the symplectic representation - indeed, the QuantumState class will infact set the above array as the X block in a PauliwordOp, with the Z block its complement.

What we are doing here is writing $|0\rangle = Z|0\rangle$ and $|1\rangle = X|0\rangle$, which ensures correct phases when multiplying the state by Pauli operators, since

$$X|0\rangle = XZ|0\rangle = -iY|0\rangle = |1\rangle,\; X|1\rangle = XX|0\rangle = |0\rangle$$
$$Y|0\rangle = YZ|0\rangle = iX|0\rangle = i|1\rangle,\; Y|1\rangle = YX|0\rangle = -iZ|0\rangle = -i|0\rangle$$
$$Z|0\rangle = ZZ|0\rangle = |0\rangle,\; Z|1\rangle = ZX|0\rangle = iY|0\rangle = -|1\rangle$$

Finally, we have $$| \vec{b} \rangle = \bigotimes_{i=1}^N \big(b_i X + (1-b_i) Z\big) | \vec{0} \rangle$$ and we may drop the zero vector and use the functionality of PauliwordOp to manipulate quantum states. In this represenation, a quantum state is stored as an operator consisting of Paulis $X, Z$, which is implicitly applied to the zero (or vacuum) state. 

In [2]:
import numpy as np
from symmer.symplectic import PauliwordOp, QuantumState

Firstly, note the correct phases under multiplication by a Pauli Y:

In [3]:
zero = QuantumState([[0]])
one = QuantumState([[1]])
Y = PauliwordOp.from_list(['Y'], [1])

print(f'{zero} -> {Y * zero}')
print(f'{one} -> {Y * one}')

 1.000+0.000j |0> ->  0.000+1.000j |1>
 1.000+0.000j |1> ->  0.000-1.000j |0>


Now, let's see what happens when we apply the Hadamard gate to the zero state:

In [4]:
psi = QuantumState([[0,0]])
had = PauliwordOp.from_list(['XZ','ZX','XX', 'ZZ'], np.ones(4)/2) # 2-qubit Hadamard gate decomposed over Paulis
eq_superposition = had * psi
print(f'Zero state: {psi}\n')
print(f'After application of the Hadamard gate:\n\n{eq_superposition}')

Zero state:  1.000+0.000j |00>

After application of the Hadamard gate:

 0.500+0.000j |00> +
 0.500+0.000j |01> +
 0.500+0.000j |10> +
 0.500-0.000j |11>


Observe that the QuantumState is represented internally by its `state_op`, a PauliwordOp object that governs its behaviour under multiplication

In [5]:
print(eq_superposition.state_op)

 0.500+0.000j ZZ +
 0.500+0.000j ZX +
 0.500+0.000j XZ +
 0.500-0.000j XX


Try evaluating expectation values for randomly generated state and Hermitian operators:

In [6]:
def random_state(num_qubits, num_terms):
    """ Generates a random normalized QuantumState
    """
    # random binary array with N columns, M rows
    random_state = np.random.randint(0,2,(num_terms,num_qubits))
    # random vector of coefficients
    coeff_vec = (
        np.random.rand(num_terms) + 
        np.random.rand(num_terms)*1j
    )
    return QuantumState(random_state, coeff_vec).normalize

def random_herm_op(num_qubits, num_terms):
    """ Generates a random PaulwordOp
    """
    # random binary array with 2N columns, M rows
    random_symp_mat = np.random.randint(0,2,(num_terms,num_qubits*2))
    # random vector of coefficients
    coeff_vec = np.random.rand(num_terms)
    coeff_vec/=coeff_vec[::-1]
    coeff_vec*=(2*np.random.randint(0,2,num_terms)-1)
    return PauliwordOp(random_symp_mat, coeff_vec)

In [7]:
N = 10 # number of qubits
M = 10 # number of terms

psi_1 = random_state(N, M)
psi_2 = random_state(N, M)

print(psi_1)
print()
print(psi_2)

 0.265+0.119j |1111101100> +
 0.345+0.030j |1011001100> +
 0.328+0.092j |1000100100> +
 0.008+0.215j |1100001011> +
 0.246+0.165j |0110011010> +
 0.260+0.349j |0110010010> +
 0.259+0.124j |1011111101> +
 0.138+0.321j |1011011111> +
 0.009+0.302j |0000001011> +
 0.076+0.232j |1111001100>

 0.088+0.018j |0010101001> +
 0.266+0.265j |1101111101> +
 0.079+0.321j |0000001011> +
 0.274+0.108j |1010000110> +
 0.174+0.316j |1000010110> +
 0.329+0.113j |1001111001> +
 0.194+0.294j |1001001011> +
 0.311+0.201j |1100101001> +
 0.194+0.100j |1101110000> +
 0.302+0.060j |0111110101>


In [8]:
psi = (psi_1 + psi_2).normalize
print(psi)
print()
print('Norm:', psi.conjugate * psi)

 0.059+0.421j |0000001011> +
 0.060+0.012j |0010101001> +
 0.175+0.235j |0110010010> +
 0.166+0.111j |0110011010> +
 0.204+0.041j |0111110101> +
 0.118+0.213j |1000010110> +
 0.222+0.062j |1000100100> +
 0.131+0.198j |1001001011> +
 0.222+0.076j |1001111001> +
 0.185+0.073j |1010000110> +
 0.233+0.020j |1011001100> +
 0.093+0.217j |1011011111> +
 0.175+0.084j |1011111101> +
 0.005+0.145j |1100001011> +
 0.210+0.136j |1100101001> +
 0.131+0.067j |1101110000> +
 0.180+0.179j |1101111101> +
 0.051+0.157j |1111001100> +
 0.179+0.080j |1111101100>

Norm: (0.9999999999999999+0j)


In [9]:
op = random_herm_op(N, 10)
print(op)
print()
print('Expectation value <psi|op|psi> =', psi.conjugate * op * psi)

 1.734+0.000j IIXXXZXYXY +
 0.955+0.000j YZZYIZYXIY +
 0.316+0.000j YIYYYYXIZZ +
-0.162+0.000j ZXXYIZYXYY +
 0.322+0.000j XZZZXXIYIX +
-3.110+0.000j ZXZXYXYYYY +
-6.173+0.000j XZXZXZIXYX +
 3.162+0.000j XIXYYYYZXY +
-1.048+0.000j YXIIYYZZZI +
-0.577+0.000j IYXXIXIZYI

Expectation value <psi|op|psi> = 0


It is also possible to convert from a $2^N$ dimensional statevector over $N$ qubits to a `QuantumState` object, via the `array_to_QuantumState` function:

In [13]:
statevector = np.zeros(2**10).reshape([-1,1])
statevector[0]   = np.sqrt(1/6)
statevector[250] = np.sqrt(2/6)
statevector[500] = np.sqrt(3/6)
psi = QuantumState.from_array(statevector)
print(psi)
print(psi.conjugate * psi)

 0.408+0.000j |0000000000> +
 0.577+0.000j |0011111010> +
 0.707+0.000j |0111110100>
(1+0j)
